### Earth weather data since 1940 (max allowed) from Open-Meteo's API


#### Locations considered

In order to get a good perspective on Earth's weather evolution, we looked into a variety of locations with different climate conditions. Listed below are the locations, their coordinated and overall description of the conditions:

1. Cold Weather: Oymyakon, Russia
Coordinates: 63.4608° N, 142.7850° E
Oymyakon is a village in Siberia known as one of the coldest inhabited places on Earth. Temperatures can drop well below -50°C (-58°F).

2. Desert: Sahara Desert, Africa
Coordinates: 25°N, 13°E (Approximate center)
The Sahara is the world's largest hot desert, covering a vast expanse of North Africa. Temperatures are scorching hot during the day and plummet at night.


3. Rainforest: Amazon Rainforest, Brazil
Coordinates: 3°S, 60°W (Approximate center)
The Amazon is the world's largest rainforest, known for its incredible biodiversity and high levels of rainfall. It has a hot and humid climate year-round.

4. Temperate: London, United Kingdom
Coordinates: 51.5074° N, 0.1278° W
London experiences a temperate climate with mild winters and warm (but not overly hot) summers. Rainfall is spread out throughout the year.

5. Mediterranean: Athens, Greece
Coordinates: 37.9838° N, 23.7275° E
Athens has a typical Mediterranean climate: hot, dry summers and mild, wetter winters.

6. Tropical Savanna: Serengeti National Park, Tanzania
Coordinates: 2.3260° S, 34.8487° E
The Serengeti is famous for its vast grasslands and wildlife. It has a tropical savanna climate with distinct wet and dry seasons.

7. Mountainous: Mount Everest Base Camp, Nepal
Coordinates: 27.9861° N, 86.9226° E
Mount Everest Base Camp offers extreme high-altitude conditions. Expect frigid temperatures, even in summer, along with snow and ice.

8. Coastal: Sydney, Australia
Coordinates: 33.8688° S, 151.2093° E
Sydney enjoys a temperate coastal climate with warm summers and mild winters. Its coastal location moderates temperatures.

9. Polar: McMurdo Station, Antarctica
Coordinates: 77.8460 °S, 166.6720° E
McMurdo Station is a research center located on the edge of Antarctica. It experiences a polar climate with extremely cold temperatures year-round and very little precipitation.

10. Semi-Arid: Death Valley, United States
Coordinates: 36.2425° N, 116.8258° W
Death Valley is one of the hottest and driest locations on Earth. This desert valley experiences extreme heat and holds the record for the highest recorded temperature on the planet.

11. Temperate Oceanic: Zurich, Switzerland 
Coordinates: 
Zurich doesn't typically experience extreme temperature swings. Summers are warm but not excessively hot, while winters are cool but rarely bitterly cold. Moderate rainfall and occasional extremes.

##### ____ Library Import ____

In [1]:
import openmeteo_requests                 # Open-Meteo weather API
import requests_cache                     # chaching responses to reduce repetitive requests
import pandas as pd
from datetime import datetime, timezone
import time
from retry_requests import retry

##### ____ Caching & Client ____

In [2]:
cache_session = requests_cache.CachedSession(".cache", expire_after = -1)   # caching object; doesn't expire (-1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)     # in case the request fails
openmeteo = openmeteo_requests.Client(session = retry_session)              # Open-Meteo client

##### ____ Coords to Process ____

In [3]:
coordinates = [
    {"name": "Oymyakon", "latitude": 63.4608, "longitude": 142.7850},   # Oymyakon, Russia
    {"name": "Sahara", "latitude": 25, "longitude": 13},                # Sahara Desert, Africa (Approximate center)
    {"name": "Amazon", "latitude": 3, "longitude": -60},                # Amazon Rainforest, Brazil (Approximate center)
    {"name": "London", "latitude": 51.5074, "longitude": -0.1278}       # London, United Kingdom
]

##### ____ Setup ____

In [4]:
url = "https://archive-api.open-meteo.com/v1/archive"       # base URL

In [5]:
now_utc = datetime.now(timezone.utc) 
end_date = now_utc.strftime("%Y-%m-%d")  

In [6]:
# params contains the query details 
base_params = {
    "start_date": "1940-01-01",
    "end_date": "2024-03-16",
    "hourly": ["temperature_2m", "relative_humidity_2m", "rain", "direct_radiation_instant"]
}

##### ____ Request ____

In [7]:
results = []  
hourly_dataframe = pd.DataFrame()


for coords in coordinates:
    params = base_params.copy()  # making a copy to avoid modifying the base parameters
    loc_name = coords["name"]
    params["latitude"] = coords["latitude"]
    params["longitude"] = coords["longitude"]


    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  

    # prints added to track progress
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()}m ASL (*Above Sea Level)")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()}s")

    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_rain = hourly.Variables(2).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(3).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["latitude"] = params["latitude"]
    hourly_data["longitude"] = params["longitude"]
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["rain"] = hourly_rain
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant


    results.append({
        "coordinates": coords, 
        "weather_data": hourly_dataframe.to_dict("records") 
    })

    temp_df = pd.DataFrame(hourly_data) 


    hourly_dataframe = pd.concat([hourly_dataframe, temp_df], ignore_index=True) 
    hourly_dataframe.to_json(f"../data/weather_data_{loc_name}.json", orient="records")

    time.sleep(60)    # to avoid limitations

Coordinates 63.444637298583984°N 142.77487182617188°E
Elevation 679.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
Coordinates 24.9912109375°N 12.981700897216797°E
Elevation 681.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
Coordinates 2.9876976013183594°N -60.0°E
Elevation 103.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
Coordinates 51.49384689331055°N -0.16302490234375°E
Elevation 16.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
